# Het mysterie van de dood van barones Heloisa

In het kasteel van barones Heloisa Argentoliensis heeft er zich een drama afgespeeld. Tijdens de nacht van de laatste volle maan werd de barones dood teruggevonden in haar torenkamer. Het overlijden is niet in het minst verdacht te noemen. De barones was nog jong en gezond. Er waren geen indicaties van ziekte. Het plotse overlijden had dus iedereen verrast. Er werd een onderzoek opgestart naar deze mysterieuze dood maar door een gebrek aan bewijs, werd nooit duidelijk wat er echt gebeurd is met barones Heloisa.

Geschiedkundigen vermoeden dat Heloisa vermoord werd door iemand die uit was op haar erfenis. Het testament dat naast haar gevonden werd, is waarschijnlijk geschreven door de dader en niet door Heloisa zelf. In dat testament staan vier personen die elk een deel van de erfenis krijgen. In de torenkamer van Heloisa werden verschillende briefwisselingen teruggevonden met elk van deze personen. Deze brieven zullen ons helpen om de dader te identificeren.

Kan jij met behulp van deze notebook achterhalen wie verantwoordelijk is voor de moord van Heloisa?

## Voorkennis

Om met deze notebook aan de slag te gaan, heb je een basiskennis nodig van het Python programmeren. We gebruiken in deze notebook datatypes, operatoren, structuren en functies. Ben je niet zeker dat je voldoende Python kennis hebt voor deze notebook, dan kun je terecht op [dwengo.org/python_programming](https://dwengo.org/python_programming/). Daar worden alle basisprincipes stap voor stap uitgelegd.

## Importeren van de nodige bibliotheken

Voor we starten met ons onderzoek, laden we eerst een aantal bibliotheken in. Deze bevatten voorgeprogrammeerde functies die we in de analyse nodig zullen hebben.

In [ ]:
# Importeer de nodige Python modules
import os
import string
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

## Het testament

Onderstaande code laadt het testament van Heloisa in in Python en drukt het af. Voer de volgende drie cellen uit en lees het testament van Heloisa. 

In [ ]:
# Deze functie leest de tekst in een bestand en geeft die terug als een string.
def lees_bestand(bestandsnaam):
    with open(bestandsnaam, encoding="utf-8") as f:
        inhoud = f.read()
    return inhoud

In [ ]:
# Lees de tekst in het bestand testament.txt en sla het op in de variabele testament.
testament = lees_bestand("documenten/testament.txt")

In [ ]:
# Druk de waarde van de variabele testament af.
print(testament)

Lees het testament van Heloisa. Wie krijgt welke delen van haar fortuin? Wat is de relatie tussen Heloisa en de begunstigden in haar testament?

## Briefwisselingen

Heloisa had nauw contact met de vier begunstigden in haar testament. In haar torenkamer werden verschillende brieven teruggevonden van elk van deze personen. Hieronder bekijken we de inhoud van een aantal van die brieven.

De functies in onderstaande cel zullen ons helpen om de teksten uit de verschillende brieven in te lezen in Python.

In [ ]:
# Deze functie leest de tekst in alle bestanden in een map en geeft die terug als een lijst van strings.
def lees_bestanden_in_map(mapnaam):
    bestanden = os.listdir(mapnaam)
    inhoud = []
    for bestand in bestanden:
        if bestand.endswith(".txt"):
            inhoud.append(lees_bestand(mapnaam + "/" + bestand))
    return inhoud

# Deze functie overloopt de mappen met auteurs en leest alle teksten van die auteurs in.
def overloop_mappen_in_map(mapnaam):
    auteurs = {}
    mappen = os.listdir(mapnaam)
    for map in mappen:
        if os.path.isdir(mapnaam + "/" + map):
            auteurs[map] = lees_bestanden_in_map(mapnaam + "/" + map)
    return auteurs

In [ ]:
# Lees de tekst voor elke auteurs in de map documenten en sla die op in een dictionary.
teksten_per_auteur = overloop_mappen_in_map("documenten")

Eerst bekijken we van welke auteurs we brieven hebben.

In [ ]:
# Druk de namen van de auteurs af.  
print(teksten_per_auteur.keys())

We zien dat we brieven hebben van elke begunstigde in het testament van Heloisa.
* Elisabeth: de dochter van Heloisa.
* Hugo: de oudste zoon van Heloisa.
* Johannes: de jongste zoon van Heloisa.
* Gregorius: de kannunik van de lokale kerkgemeenschap.

Laten we nu de inhoud van een aantal van de brieven bekijken.

In [ ]:
# Druk de inhoud van de eerste brief van Hugo af.
print(teksten_per_auteur["Hugo"][0])

In [ ]:
# Druk de inhoud van de vijfde brief van Gregorius af.
print(teksten_per_auteur["Gregorius"][4])

**Opdracht:** Vul onderstaande code aan zodat je de brieven kan lezen die Elisabeth naar haar moeder schreef.

In [ ]:
print(teksten_per_auteur[___][___])

## Schrijfstijl

Elke persoon heeft een eigen schrijfstijl. Door deze stijl te analyseren, kunnen we achterhalen welke teksten door een en dezelfde auteur geschreven zijn. Deze techniek kunnen we hier ook toepassen. Door de schrijfstijl van het testament te vergelijken met die van de begunstigden in het testament, kunnen we een verdachte voor de moord identificeren.

Hier gebruiken we een eenvoudige techniek om de schrijfstijl van een auteur te bepalen. Dat doen we door te tellen hoe vaak de schrijver bepaalde eenvoudige woorden zoals de, het, u, jij, ... gebruikt in hun tekst. Het aantal keer dat elk van deze woorden voorkomt in een tekst van een auteur noemen we de vingerafdruk van deze auteur.

Om de vingerafdruk van een auteur te kunnen bepalen, moeten we de volgende stappen uitvoeren.

* **Lowercasing**: Alle hoofdletters in de tekst worden vervangen door kleine letters. Lowercasing is nodig omdat we gaan tellen hoeveel keer elk woord voorkomt en we het woord met en zonder hoofdletter op dezelfde manier willen tellen.
* **Tokenisering:**: Alle zinnen worden in betekenisvolle eenheden of 'tokens' gesplitst, zoals woorden en leestekens. Deze splitsing gebeurt op basis van de aanwezige spaties in de zinnen; daarom zullen de woorden van elkaar moeten gescheiden zijn door een spatie.
* **Filteren**: Om een vingerafdruk van de tekst te maken, kijken we enkel naar het gebruik van veelvoorkomende woorden (bv. de, het, u, uw, je, jij, ...). Woorden die niet vaak voorkopen filteren we dus weg uit de tekst.
* **Frequentie-analyse**: Hier tellen we hoe vaak deze bepaalde woorden gebruikt. Zo krijgen we een vingerafdruk van de tekst die we kunnen vergelijken met die van andere teksten.
* **Normaliseren**: Om ervoor te zorgen dat we teksten van verschillende lengte met elkaar kunnen vergelijken, normaliseren we de frequentie van elk woord.

## De vingerafdruk van het testament

Eerst bepalen we de vingerafdruk van het testament. Deze vingerafdruk kunnen we dan later vergelijken met die van de teksten van de verschillende auteurs.

### Lowercasing

Hier vervangen we in het testament hoofdletters door kleine letters.

In [ ]:
# Druk het testament af
print(testament)

# Zet hoofdletters om naar kleine letters.
testament = testament.lower()

# Druk het testament nog eens af.
print(testament)

### Tokenizering

Hier splitsen we de tekst op in afzonderlijke woorden. Daarvoor verwijderen we eerst de leestekens, daarna splitsten we de tekst overal waar een spatie staat. Zo krijgen we een lijst van afzonderlijke woorden.

In [ ]:
# Eerst verwijderen we de leestekens uit de tekst.
def verwijder_leestekens(tekst):
    leestekens = string.punctuation
    tekst_zonder_leestekens = ""
    for karakter in tekst:
        if karakter not in leestekens:
            tekst_zonder_leestekens = tekst_zonder_leestekens + karakter
            
    return tekst_zonder_leestekens

In [ ]:
# Verwijder de leestekens en splits de tekst in woorden.
testament = verwijder_leestekens(testament).split()

In [ ]:
print(testament)

### Filteren

Om een vingerafdruk van een tekst te maken, gebruiken we enkel een vaste verzameling van veelvoorkomende woorden. In de volgende codecel, filteren we elke tekst en houden we enkel de woorden over die nuttig zijn voor de vingerafdruk.

In [ ]:
# Dit is de lijst met woorden die we gaan tellen.
vingerafdruk_woorden = ["de", "het", "een", "en", "in", "op", "van", "voor", "achter", "onder", "boven", "tussen", "tegen", "met", "zonder", "bij", "door", "naar", "uit", "over", "langs", "rond", "om", "naast", "binnen", "buiten", "ik", "jij", "u", "wij", "jullie", "zij", "mijn", "jouw", "uw", "zijn", "haar", "onze", "hun", "die", "dat", "wie", "wat", "welke", "men"]

# Maak een nieuwe lijst met enkel de woorden die in de vingerafdruk kunnen voorkomen.
testament = [woord for woord in testament if woord in vingerafdruk_woorden]

In [ ]:
print(testament)

### Frequentie-analyse

Nu we enkel nog woorden over hebben die van belang zijn voor onze vingerafdruk, tellen we hoe vaak elk van deze woorden voorkomt in de tekst.

In [ ]:
# Bereken de frequentie van elk woord in het testament.
frequentie_testament = {}
for woord in vingerafdruk_woorden:
    frequentie_testament[woord] = testament.count(woord)

In [ ]:
print(frequentie_testament)

### Normaliseren

Omdat het aantal woorden dat we tellen afhankelijk is van de lengte van de tekst is het moeilijk om teksten van verschillende lengte te vergelijken. Om toch teksten van verschillende lengte te kunnen vergelijken, kunnen we de frequentie van elk woord normaliseren. Normalisatie is een stap die heel vaak gebruikt wordt in AI-systemen en nodig is om inputs op een gelijkaardig "niveau" te brengen. Hier normaliseren we het aantal voorkomens van een woord door het te delen door het totaal aantal woorden dat we geteld hebben. We zetten de frequenties dus eigenlijk om naar percentages.

In [ ]:
# Deze functie normaliseert de frequentie van elk woord in een tekst.
def normaliseer(frequentie):
    som = sum(frequentie.values())
    for woord in frequentie:
        if som > 0:
            frequentie[woord] = frequentie[woord] / som
        else:
            frequentie[woord] = 0
    return frequentie

In [ ]:
frequentie_testament = normaliseer(frequentie_testament)

In [ ]:
print(frequentie_testament)

We kunnen deze frequenties ook weergeven in een grafiek. Zo krijgen we een visuele weergave van onze vingerafdruk.

In [ ]:
# Maak voor elke auteur en voor het testament een histogram van de frequenties van de woorden.
x = np.arange(len(vingerafdruk_woorden))
breedte = 1

fig, ax = plt.subplots(figsize=(20, 10))
rects_testament = ax.bar(x - breedte/2, list(frequentie_testament.values()), breedte, label='Testament')
ax.set_xticks(x)
ax.set_xticklabels(vingerafdruk_woorden, rotation=45, ha="right")
ax.legend()
plt.show()

## Toepassen op de brieven

We hebben nu alle stappen doorlopen om een vingerafdruk van onze tekst te maken. Deze zelfde stappen kunnen we nu toepassen op de brieven van de kennissen van Heloisa. Om dat op een overzichtelijke manier te doen, schrijven we een functie die de vingerafdruk van een stuk tekst kan bepalen.

In [ ]:
def zet_tekst_om_naar_vingerafdruk(tekst):
    tekst = tekst.lower() # Lowercasing
    tekst = verwijder_leestekens(tekst).split() # Verwijder leestekens en splits in woorden
    tekst = [woord for woord in tekst if woord in vingerafdruk_woorden] # Enkel de woorden die in de vingerafdruk kunnen voorkomen
    # Bereken de frequentie van elk woord in de tekst
    frequentie = {}
    for woord in vingerafdruk_woorden:
        frequentie[woord] = tekst.count(woord)
    frequentie = normaliseer(frequentie) # Normaliseer de frequentie
    return frequentie

We maken ook een functie die de vingerafdruk kan visualiseren.

In [ ]:
def visualiseer_vingerafdruk(frequentie, titel):
    x = np.arange(len(vingerafdruk_woorden))
    breedte = 1

    fig, ax = plt.subplots(figsize=(20, 10))
    rects = ax.bar(x - breedte/2, list(frequentie.values()), breedte, label=titel)
    ax.set_xticks(x)
    ax.set_xticklabels(vingerafdruk_woorden, rotation=45, ha="right")
    ax.legend()
    ax.title.set_text(titel)
    plt.show()

Hieronder zetten we de eerste brief van Elisabet om naar een vingerafdruk en tonen die op het scherm.

In [ ]:
vingerafdruk = zet_tekst_om_naar_vingerafdruk(teksten_per_auteur["Elisabeth"][0])
print(vingerafdruk)
visualiseer_vingerafdruk(vingerafdruk, "Elisabeth 0")

In [ ]:
vingerafdruk = zet_tekst_om_naar_vingerafdruk(teksten_per_auteur["Elisabeth"][2])
print(vingerafdruk)
print(frequentie_testament)

**Opdracht:** Pas onderstaande code aan en bekijk zo de vingerafdrukken van de volgende brieven:
* De tweede brief van Hugo.
* De derde brief van Johannes.
* De vijfde brief van Gregorius.

In [ ]:
vingerafdruk = zet_tekst_om_naar_vingerafdruk(teksten_per_auteur[___][___])
print(vingerafdruk)
visualiseer_vingerafdruk(vingerafdruk, ___)

## Afstand bepalen

Om te achterhalen welke auteur een stijl heeft die overeenkomt met de stijl in het testament, vergelijken we de vingerafdruk van elke brief met deze van het testament. Dien doen we door de euclidische afstand te berekenen tussen de vingerafdruk van de brief en deze van het testament.

Om de afstand te kunnen bepalen, moeten we eerst elke brief omzetten naar een vingerafdruk. Dat kun je doen met onderstaande code.

In [ ]:
# Hier overlopen we alle teksten per auteur en zetten we die om naar vingerafdrukken.
vingerafdrukken = {}
for auteur in teksten_per_auteur:
    vingerafdrukken[auteur] = []
    for i, tekst in enumerate(teksten_per_auteur[auteur]):
        vingerafdrukken[auteur].append(zet_tekst_om_naar_vingerafdruk(tekst))

Nu kunnen we de afstand berekenen tussen de vingerafdruk van elke brief en die van het testament.

In [ ]:
# Bereken de afstand tussen elke vingerafdruk en het testament.
afstanden = {}
for auteur in vingerafdrukken:
    afstanden[auteur] = []
    for vingerafdruk in vingerafdrukken[auteur]:
        afstand = 0
        for woord in vingerafdruk:
            afstand += abs(vingerafdruk[woord] - frequentie_testament[woord])
        afstanden[auteur].append(afstand)

In [ ]:
# Druk de afstanden af.
for auteur in afstanden:
    print(auteur, afstanden[auteur])

**Opdracht:**: Kan je uit bovenstaande uitvoer aflezen welke auteur een stijl heeft die het dichtste ligt bij de stijl van het testament?

## Clusteren

Op basis van de afstanden kan het moeilijk zijn om te zien hoe ver de teksten van elke auteur van elkaar liggen. Om te zien welke teksten bij elkaar horen en welke niet, kan je clustering technieken gebruiken. Deze technieken zoeken structuren in data. Concreet groepeert deze techniek de elementen in de dataset die dicht bij elkaar liggen. Hieronder gebruiken we de K-means clusteringstechniek. Bij deze techniek moeten we vooraf opgeven hoeveel groepen we verwachten. Omdat we hier teksten van vijf auteurs hebben, kiezen we hier voor vijf groepen.

Om de gegevens te kunnen clusteren, moeten we deze omzetten naar een correct formaat. Onderstaande cel voegt de vingerafdruk van het testament en van de verschillende auteurs samen in een matrix. Daarnaast slaan we ook de namen van de auteurs op in een lijst. Deze lijst kunnen we gebruiken om te controleren hoe goed onze clustering is.

In [ ]:
frequenties = []
labels = []
frequenties.append(list(frequentie_testament.values()))
labels.append("Testament")
for auteur in vingerafdrukken:
    for frequentie in vingerafdrukken[auteur]:
        frequenties.append(list(frequentie.values()))
        labels.append(auteur)

De onderstaande cel zal onze data clusteren in vijf groepen.

In [ ]:
# Cluster de verschillende teksten van de auteurs en het testament.
kmeans = KMeans(n_clusters=5)
kmeans.fit(frequenties)

Nu kunnen we onze datapunten overlopen en afdrukken bij welke cluster horen.

In [ ]:
# Druk de clusters af.
for i, label in enumerate(labels):
    print(f'Tekst: {label}, Cluster: {kmeans.labels_[i]}')

Hierboven zie je telkens de naam van de auteur, elke van deze namen komt overeen met een van de teksten van die auteur. Ernaast zie je het cijfer van de groep waartoe deze tekst behoort. Merk op dat het resultaat van K-means niet altijd hetzelfde is. Dat komt omdat het algoritme begint met willekeurige middelpunten van de clusters. De keuze van deze punten heeft dus invloed op het uiteindelijke resultaat. Je kan bovenstaande twee cellen best een aantal keer uitvoeren. Zo zal je een idee krijgen van hoe de clustering kan variëren.

We kunnen het resultaat van onze clustering ook visualiseren. Om dat op een 2D vlak te kunnen doen moeten we onze vingerafdrukken, die nu uit een reeks van meerdere getallen bestaan, omzetten naar 2D coördinaten. Een techniek die we daarvoor kunnen gebruiken is PCA ofwel principle component analysis. Deze techniek zal onze vingerafdrukken (vectoren van lengte 45) omzetten naar 2D coördinaten.

In [ ]:
# Gebruik PCA om onze vingerafdrukken om te zetten naar 2D coördinaten.  
pca = PCA(n_components=2)
pca.fit(frequenties)
frequenties_pca = pca.transform(frequenties)

Eens we de omzetting gedaan hebben, kunnen we de vingerafdrukken weergeven in een spreidingsdiagram. Tegelijk kunnen we ook de nummers van de clusters gebruiken om de punten op het diagram een kleur te geven die bij die cluster hoort. Zo kunnen we zien welke teksten er volgens het K-means algoritme samen horen.

In [ ]:
# Geef de vingerafdrukken weer in een spreidingsdiagram.
plt.scatter(frequenties_pca[:, 0], frequenties_pca[:, 1], c=kmeans.labels_)
for i, label in enumerate(labels):
    plt.text(frequenties_pca[i, 0], frequenties_pca[i, 1], label)
plt.show()

Uit bovenstaande resultaten kunnen we opmaken dat de schrijfstijl van het testament het meeste lijkt op die van Elisabeth. Elisabeth wordt hierdoor dus onze hoofdverdachte. Toch ligt de schrijfstijl van het testament ook nog relatief dicht bij dat van Heloisa zelf. Er is dus een indicatie dat het testament door Elisabeth geschreven werd maar daar zijn we dus nog niet zeker van. Het is ook niet duidelijk wat het motief van Elisabeth geweest zou zijn. Om meer duidelijkheid te krijgen over het overlijden van Heloisa zal je nog andere bewijsstukken moeten analyseren.

TIP: Letters worden door de computer ook voorgesteld als cijfers. 